In [1]:
data_var = '2024-01-20'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13302,2024-01-20,Brasil Nbb,11:00,Botafogo,União Corinthians,1.48,2.53,160.5,1.87,1.79,-5.5,2.01,1.72,Wpn30Fvs,0.675676,0.395257,0.534759,0.558659,0.070933,0.6,0.4,NaN,NaN,251.720,97.508441,0.387369,262.022,160.804090,0.613705,215.386,348.014,123.12,443.75,0.0,0.0,0.0,0.0,0.370305,0.030912,0.109952,0.70,0.140,3.428571,0.369867,0.3,-0.069867,-1.14,-0.228,-6.710526,0.000000,0.0,0.000000
13303,2024-01-20,Brasil Nbb,17:00,R10 Score Vasco da Gama,Caxias do Sul,1.25,3.69,145.5,1.82,1.84,-9.5,1.96,1.75,Ayo7aZgm,0.800000,0.271003,0.549451,0.543478,0.071003,0.4,0.4,NaN,NaN,184.524,96.161146,0.521131,287.308,164.142643,0.571312,137.756,226.408,116.64,180.40,0.0,0.0,0.0,0.0,0.698518,0.007728,0.080050,1.30,0.260,0.961538,0.000000,0.0,0.000000,1.06,0.212,12.688679,0.332202,0.4,0.067798
13304,2024-01-20,Espanha Acb,14:00,Gran Canaria,Murcia,1.33,3.38,163.5,1.85,1.91,-8.5,2.01,1.76,jaY9KyD0,0.751880,0.295858,0.540541,0.523560,0.047738,0.4,0.6,NaN,NaN,170.690,75.528622,0.442490,154.528,48.556135,0.314222,110.632,127.352,98.04,129.21,0.0,0.0,0.0,0.0,0.615528,0.022567,0.093781,-0.26,-0.052,-6.346154,0.000000,0.0,0.000000,-0.18,-0.036,-66.111111,0.000000,0.0,0.000000
13305,2024-01-20,Espanha Acb,14:00,Obradoiro CAB,Tenerife,2.43,1.57,160.5,1.88,1.92,2.5,2.03,1.73,dxZ5LHcf,0.411523,0.636943,0.531915,0.520833,0.048465,0.0,0.8,NaN,NaN,327.650,238.892541,0.729109,175.372,99.518484,0.567471,0.000,255.060,173.01,150.48,0.0,0.0,0.0,0.0,0.304056,0.014886,0.112836,0.00,0.000,inf,0.000000,0.0,0.000000,1.43,0.286,1.993007,0.000000,0.0,0.000000
13306,2024-01-20,Espanha Acb,16:45,Joventut,Unicaja,2.50,1.54,162.5,1.83,1.93,2.5,2.10,1.68,nRRIIFsD,0.400000,0.649351,0.546448,0.518135,0.049351,0.4,0.4,NaN,NaN,190.754,49.004775,0.256900,132.966,42.228650,0.317590,159.740,186.188,129.20,100.62,0.0,0.0,0.0,0.0,0.336051,0.037612,0.157135,1.75,0.350,4.285714,0.000000,0.0,0.000000,5.39,1.078,0.500928,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13688,2024-01-20,Hungria Nacional,14:00,Kormend,DEAC,1.45,2.62,148.5,1.85,1.85,-6.5,2.08,1.67,tW57yl1Q,0.689655,0.381679,0.540541,0.540541,0.071335,0.6,0.4,NaN,NaN,149.676,58.299733,0.389506,145.032,43.106350,0.297220,125.510,236.892,126.04,190.80,0.0,0.0,0.0,0.0,0.406543,0.000000,0.154621,-0.70,-0.140,-3.214286,0.000000,0.0,0.000000,-0.57,-0.114,-14.210526,0.000000,0.0,0.000000
13689,2024-01-20,Hungria Nacional Feminino,14:00,Cegledi F,Szekszard F,11.10,1.02,137.5,1.80,1.86,30.5,2.01,1.71,jyx8B90m,0.090090,0.980392,0.555556,0.537634,0.070482,0.0,0.0,NaN,NaN,654.050,316.504471,0.483915,167.884,67.823701,0.403991,0.000,0.000,401.40,80.80,1.0,0.0,0.0,0.0,1.176178,0.023184,0.114049,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
13690,2024-01-20,Hungria Nacional Feminino,14:00,Sopron F,Vasas Csata F,1.04,9.85,138.5,1.85,1.85,-26.5,2.00,1.72,YaRCATFg,0.961538,0.101523,0.540541,0.540541,0.063061,0.0,0.0,NaN,NaN,134.556,48.898606,0.363407,244.132,218.667061,0.895692,0.000,0.000,118.34,81.00,0.0,0.0,0.0,0.0,1.144097,0.000000,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
13691,2024-01-20,Lituânia King Mindaugas Cup,13:30,Rytas,Juventus,1.34,3.01,175.5,1.87,1.79,-11.5,2.09,1.63,dOighnN0,0.746269,0.332226,0.534759,0.558

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Portland Trail Blazers,Indiana Pacers,237.5,1.88,0.9759,Over
1,00:30,Eua Nba,Los Angeles Lakers,Brooklyn Nets,224.5,1.82,0.8227,Over
2,21:30,Eua Nba,Atlanta Hawks,Cleveland Cavaliers,231.5,1.93,0.9425,Over
3,21:30,Eua Nba,New York Knicks,Toronto Raptors,228.5,1.93,0.5874,Over
4,10:00,Rússia Liga Vtb United,Saratov,Samara,160.5,1.82,0.9778,Over
5,18:00,Eua Ncaa,Abilene Christian,Texas Arlington,154.5,1.91,0.8404,Over
